LangChainMemory : 이전 대화 내용을 기억햇서 문맥을 유지하는 역할 Langchain 0,   
3X부터는 LCEL 기반으로 체인을 구성, RunnableWithMessageHistory,    
ChatMessageHistory 등의 컴포넌트를 활용해서 세션별 대화 기록을 관리, 대화가 장기화 될 경우       
요약 메모리를 도입해서 과거 대화를 LLM으로 요약하고 축약된 형태로 저장해서 프롬프트의 길이문제를 해결  


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [8]:
from langchain_core.chat_history import InMemoryChatMessageHistory

# 메모리 객체 생성
history = InMemoryChatMessageHistory()
# 필요한 멤버 추가
history.add_user_message("안녕하세요 제 이름은 홍길동입니다.")
history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

human: 안녕하세요 제 이름은 홍길동입니다.
ai: 안녕하세요 홍길동님, 무엇을 도와드릴까요?


In [ ]:
## Redis를 이용한 채팅 기록 저장
from langchain_redis import RedisChatMessageHistory
import os
Redis_URL = os.getenv("REDIS_CHAT_URL", "redis://localhost:6379")
session_id = "uset_123"
# Redis 메모리 객체 생성
history = RedisChatMessageHistory(session_id= session_id , redis_url=Redis_URL)
# 필요한 멤버 추가
history.add_user_message("안녕하세요 제 이름은 홍길동입니다.")
history.add_ai_message("안녕하세요 홍길동님, 무엇을 도와드릴까요?")
# 현재까지의 대화 내용 확인
for msg in history.messages:
    print(f"{msg.type}: {msg.content}")

# 다음 프로젝트 때 이어서 할 예정


In [ ]:
# 세션 기반 다중 사용자 메모리 구조 구현
# 핵심 : session_id를 키로 하는 메모리 저장소 만들고 사용자의 대화는 키별로 저장한다.
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뛰어난 한국어 상담 챗봇입니다. 질문에 친절하고 자세히 답변해주세요"),
    MessagesPlaceholder(variable_name="history"), 
    # history키로 전달된 메세지 목록은 체인 실행시 해당 위치에 넣겠다는 의미
    ("human", "{input}"), 
])

llm = ChatOpenAI(temperature=0.0, model="gpt-4o-mini")

In [11]:
# LCEL 체인 생성
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

In [12]:
# 세션별 메모리 저장소를 딕셔너리로 만들고, 존재하지 않는 새로운 세션 id가 들어오면 InmemoryChatMessageHistory를 생성
# get_session_history를 구현

In [35]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# 세션 id -> 대화 기록 객체 매핑
store = {}
def get_session_history(session_id: str):
    '''"세션 ID"에 해당하는 대화 기록 객체를 반환합니다.(없으면 새로 생성)'''
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 메모리를 통합한 체인 래퍼 생성
chatbot = RunnableWithMessageHistory(
    runnable = chain,                 # runnable (첫 번째)
    get_session_history = get_session_history,   # message_history_getter (두 번째)
    input_messages_key='input',
    history_messages_key='history'
)


In [43]:
import logging

logging.getLogger("httpx").setLevel(logging.WARNING)

In [44]:
# 두개의 세션을 번갈아가면서 대화
sessions = ['user_a', 'user_b']
questions = [
    '안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?', # user_a 첫번째 질문
    '안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?', # user_b 첫번째 질문
    '저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?', # user_a 두번째 질문
    '저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?' # user_b 두번째 질문
]
for i, question in enumerate(questions):
    session_id = sessions[i % 2]  # 세션 ID를 번갈아가며 사용
    result = chatbot.invoke({'input': question}, config = {'configurable': {'session_id': session_id}})
    print(f'[{session_id}] 질문: {question}')
    print(f'[{session_id}] 챗봇: {result}\n')

[user_a] 질문: 안녕하세요, 저는 홍길동입니다. 당신은 누구신가요?
[user_a] 챗봇: 안녕하세요, 홍길동님! 저는 여러분의 질문에 답변하고 도움을 드리기 위해 만들어진 챗봇입니다. 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!

[user_b] 질문: 안녕하세요, 저는 이순신입니다. 당신은 어떤 일을 하시나요?
[user_b] 챗봇: 안녕하세요, 이순신님! 저는 여러분의 질문에 답변하고, 정보 제공, 상담 등을 도와주는 챗봇입니다. 다양한 주제에 대해 이야기할 수 있으니 궁금한 점이나 도움이 필요하신 부분이 있다면 언제든지 말씀해 주세요!

[user_a] 질문: 저는 프로그래밍을 배우고 있습니다. 당신은 어떤 일을 하시나요?
[user_a] 챗봇: 프로그래밍을 배우고 계시다니 정말 멋지네요! 저는 여러분의 질문에 답변하고, 다양한 주제에 대한 정보를 제공하는 역할을 합니다. 프로그래밍 관련 질문, 알고리즘, 데이터 구조, 특정 언어에 대한 설명 등 여러 가지 도움을 드릴 수 있습니다. 궁금한 점이 있다면 언제든지 말씀해 주세요!

[user_b] 질문: 저는 역사에 관심이 많습니다. 당신은 어떤 분야에 관심이 있나요?
[user_b] 챗봇: 역사에 관심이 많으시군요! 역사적인 사건이나 인물에 대해 이야기하는 것은 정말 흥미로운 주제입니다. 저는 특정한 관심 분야가 없지만, 다양한 주제에 대한 정보를 제공할 수 있습니다. 역사, 과학, 기술, 문화 등 여러 분야에 대해 질문하시면 최대한 자세히 답변해 드리겠습니다. 어떤 역사적 사건이나 인물에 대해 이야기해 보고 싶으신가요?



In [45]:
result = chatbot.invoke({'input': "저는 철수예요, 반갑습니다."}, config = {'configurable': {'session_id': 'user_c'}})
print(f'[user_c] 질문: "저는 철수예요, 반갑습니다."')
print(f'[user_c] 챗봇: {result}\n')

[user_c] 질문: "저는 철수예요, 반갑습니다."
[user_c] 챗봇: 안녕하세요, 철수님! 다시 만나서 반갑습니다. 오늘은 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 도움이 필요하신 부분이 있다면 말씀해 주세요!



In [46]:
result = chatbot.invoke({'input': "저는 누구라고요?"}, config = {'configurable': {'session_id': 'user_a'}})
result

result = chatbot.invoke({'input': "저는 누구라고요?"}, config = {'configurable': {'session_id': 'user_c'}})
result

'철수님이라고 하셨습니다! 맞나요? 혹시 다른 질문이나 이야기하고 싶은 것이 있으신가요? 언제든지 말씀해 주세요!'

# 요약 메모리 구현(대화내용 자동 요약)
긴 대화내용을 모두 프롬프트에 기록하는것은 비효율적 -> 프롬프트의 길이 제한에 걸릴 가능성이 있음  
Conversation Summary Memory  
0.3x 버전에서는 직접 요약용 체인을 만들어서 ChatMessageHistory에 적용

- 어떻게 요약?  
  - 일정 길이 이상으로 대화가 누적되면, 과거 대화를 요약해서 핵심내용만 남김
  - 요약결과를 메모리에 시스템 메세지 등으로 저장 ->메모리 절약
  - 새로운 사용자 입력시 요약된 맥락 - 최근 몇 메시지만 참고해서 llm 전달



In [48]:
# 요약용 프롬프트 탬플릿
summary_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 대화 요약 전문가입니다. 대화의 주요 내용을 간결하게 요약해 주세요"),
    ("human", "{conversation}"), # 전체 대화냐용을 하나의 문자열로 전달
])  
# LCEL
summary_chain = summary_prompt | llm | StrOutputParser()

In [50]:
store.keys()

dict_keys(['user_a', 'user_b', 'user_c'])

In [52]:
# user_d 세션의 대화내용을 기록 긴 대화 생성
long_queries = [
    "안녕, 오늘 우리 뭐하려고 했지?",
    "아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?",
    '그 회의에 누가 참석하는지 기억나니',
    '단위프로젝트 진행 상황도 공유해야 할까?',
    '최근에 이야기 했던 새로운 기능에 대한 아이디어가 있어'
]

session_id = 'user_d'
for q in long_queries:
    answer = chatbot.invoke({'input': q}, config = {'configurable': {'session_id': session_id}})
    print(f'[{session_id}] 질문: {q}')
    print(f'[{session_id}] 챗봇: {answer}\n')

[user_d] 질문: 안녕, 오늘 우리 뭐하려고 했지?
[user_d] 챗봇: 안녕하세요! 오늘 어떤 이야기를 나누고 싶으신가요? 궁금한 점이나 하고 싶은 주제가 있다면 말씀해 주세요. 함께 이야기해보아요!

[user_d] 질문: 아 맞다 내일 회의자료 준비해야지, 회의는 몇시지?
[user_d] 챗봇: 회의 시간이 기억나지 않으신가요? 보통 회의 일정은 이메일이나 캘린더에 기록되어 있을 텐데요. 확인해보시고, 만약 찾기 어렵다면 회의 주최자에게 직접 문의해보시는 것도 좋은 방법입니다. 회의 준비에 도움이 필요하시면 말씀해 주세요! 어떤 자료를 준비해야 하는지에 대해서도 도와드릴 수 있습니다.

[user_d] 질문: 그 회의에 누가 참석하는지 기억나니
[user_d] 챗봇: 회의 참석자에 대한 정보는 제가 알 수 없지만, 보통 회의 초대장이나 이메일에 참석자 목록이 포함되어 있을 거예요. 만약 그 정보를 찾기 어렵다면, 회의 주최자에게 확인해보시는 것이 좋습니다. 참석자 목록이 필요하시다면, 어떤 내용을 준비해야 할지에 대해서도 도와드릴 수 있으니 말씀해 주세요!

[user_d] 질문: 단위프로젝트 진행 상황도 공유해야 할까?
[user_d] 챗봇: 네, 단위 프로젝트 진행 상황을 공유하는 것은 좋은 아이디어입니다. 회의에서 프로젝트의 현재 상태, 진행된 작업, 남은 일정, 그리고 발생한 문제점 등을 공유하면 참석자들이 프로젝트에 대한 이해를 높이고, 필요한 지원이나 피드백을 받을 수 있습니다. 

또한, 진행 상황을 공유함으로써 팀원 간의 협업을 촉진하고, 프로젝트의 목표를 다시 한 번 확인할 수 있는 기회가 될 것입니다. 어떤 내용을 포함할지 고민 중이시라면, 구체적인 사항에 대해 말씀해 주시면 더 도움을 드릴 수 있습니다!

[user_d] 질문: 최근에 이야기 했던 새로운 기능에 대한 아이디어가 있어
[user_d] 챗봇: 새로운 기능에 대한 아이디어가 있다니 정말 흥미롭네요! 어떤 아이디어인지 자세히 말씀해 주시면 좋겠습니다. 기능의 목적, 예

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
# 요약 대상 대화내용 추출(마지막 QA 쌍 제외한 이전 내용)
message = store[session_id].messages

if len(message) > 2:
    original_dialog = '\n'.join(f'{msg.type.upper()} : {msg.content}' for msg in message[: -2])
else:
    original_dialog = '\n'.join(f'{msg.type.upper()} : {msg.content}' for msg in message)

# llm으로 요약 생성
summary = summary_chain.invoke({'conversation': original_dialog})
print("== 요약 내용 ==")
print(summary)
# 기존 메모리를 요약으로 교체 : 이전내용 요약본 
new_history = InMemoryChatMessageHistory()
new_history.messages.append(SystemMessage(content=f"요약 : {summary}"))
# 최근 대화의 마지막 QA쌍 복원
if len(message) > 2:
    last_user_msg = message[-2]
    last_ai_msg= message[-1]
    if isinstance(last_user_msg, HumanMessage) :
        new_history.add_user_message(last_user_msg.content)
    else:
        new_history.messages.append(last_user_msg)
    if isinstance(last_ai_msg, AIMessage) :
        new_history.add_ai_message(last_ai_msg.content)
    else:
        new_history.messages.append(last_ai_msg)

# 기존 메모리 객체를 새로운 대화 기록으로 교체
store[session_id] = new_history

== 요약 내용 ==
사용자는 내일 회의 자료 준비에 대해 언급하며 회의 시간과 참석자에 대해 질문했습니다. AI는 이메일이나 캘린더에서 확인하라고 조언하고, 프로젝트 진행 상황을 공유하는 것이 팀 협업에 도움이 된다고 제안했습니다. AI는 추가적인 도움을 제공할 준비가 되어 있다고 덧붙였습니다.


In [56]:
len(store[session_id].messages)

3